https://s3-us-west-2.amazonaws.com/allennlp/models/srl-model-2017.09.05.tar.gz

In [65]:
from nltk.corpus import verbnet, wordnet

from allennlp.service.predictors import SemanticRoleLabelerPredictor
from allennlp.models.archival import load_archive

from SPARQLWrapper import SPARQLWrapper, JSON

In [179]:
import spacy
from spacy.tokens import Doc, Span, Token

nlp = spacy.load('en')

# Semantic role labeling

In [2]:
archive = load_archive('/home/drqa/Projects/nl-to-sparkle/srl-model-2017.09.05.tar.gz')
predictor = SemanticRoleLabelerPredictor.from_archive(archive, "semantic-role-labeling")

Did not use initialization regex that was passed: stacked_encoder\\\\._module.layer_0\\\\.input_linearity\\\\.weight
Did not use initialization regex that was passed: stacked_encoder\\\\._module.layer_.*bias
Did not use initialization regex that was passed: stacked_encoder\\\\._module\\\\.layer_0\\\\.state_linearity\\\\.weight


In [180]:
class SRLTagger(object):
    name = 'SRLTagger'
    
    def __init__(self, vocab, srl_predictor):
        self.vocab = vocab
        self.srl_predictor = srl_predictor
        
        Token.set_extension('srl_tag', default='O')
        #Span.set_extension('srl_tag', default='O')
        
    def __call__(self, doc):
        predictions = self.srl_predictor.predict_json({'sentence': doc.text})
        
        if len(predictions['verbs']):
            for i, token in enumerate(doc):
                if token.text == predictions['tokens'][i]:
                    token._.set('srl_tag', predictions['verbs'][0]['tags'][i])
        return doc
    
srl_tagger = SRLTagger(nlp, predictor)
nlp.add_pipe(srl_tagger, last=True)

In [ ]:
doc = nlp('What cartel trades guns illegally?')

In [187]:
for token in doc:
    print(token, token._.srl_tag, sep='\t')

What	O
cartel	B-ARG0
trades	B-V
guns	B-ARG1
illegally	B-ARGM-MNR
?	O


In [148]:
srl_output = predictor.predict_json({'sentence':'What cartel trades guns illegally?'})
srl_output

{'tokens': ['What', 'cartel', 'trades', 'guns', 'illegally', '?'],
 'verbs': [{'description': 'What [ARG0: cartel] [V: trades] [ARG1: guns] [ARGM-MNR: illegally] ?',
   'tags': ['O', 'B-ARG0', 'B-V', 'B-ARG1', 'B-ARGM-MNR', 'O'],
   'verb': 'trades'}],
 'words': ['What', 'cartel', 'trades', 'guns', 'illegally', '?']}

In [38]:
predictor.predict_json({'sentence':'What is the capital of Bolivia?'})

{'tokens': ['What', 'is', 'the', 'capital', 'of', 'Bolivia', '?'],
 'verbs': [{'description': '[ARG2: What] [V: is] [ARG1: the capital of Bolivia] ?',
   'tags': ['B-ARG2', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O'],
   'verb': 'is'}],
 'words': ['What', 'is', 'the', 'capital', 'of', 'Bolivia', '?']}

In [115]:
predictor.predict_json({'sentence':'Who is the author of A Game of Thrones?'})

{'tokens': ['Who',
  'is',
  'the',
  'author',
  'of',
  'A',
  'Game',
  'of',
  'Thrones',
  '?'],
 'verbs': [{'description': '[ARG1: Who] [V: is] [ARG2: the author of A Game of Thrones] ?',
   'tags': ['B-ARG1',
    'B-V',
    'B-ARG2',
    'I-ARG2',
    'I-ARG2',
    'I-ARG2',
    'I-ARG2',
    'I-ARG2',
    'I-ARG2',
    'O'],
   'verb': 'is'}],
 'words': ['Who',
  'is',
  'the',
  'author',
  'of',
  'A',
  'Game',
  'of',
  'Thrones',
  '?']}

# WordNet and VerbNet Corpus

In [147]:
asdf = wordnet.synsets('cartel')[0]
asdf.

Synset('trust.n.04')

In [109]:
asdf = vnclass[0][0]
asdf.get

TypeError: Required argument 'key' (pos 1) not found

In [116]:
verbnet.wordnetids(vnclass)

['change%2:30:05',
 'change%2:30:04',
 'exchange%2:30:00',
 'exchange%2:41:00',
 'trade%2:40:00',
 'trade%2:40:02']

In [139]:
asdf = vnclass[0][0]
asdf.items()

[('name', 'change'), ('wn', 'change%2:30:05 change%2:30:04')]

In [100]:
vnclass = verbnet.vnclass(verbnet.classids('trade')[0])
for i in vnclass:
    for j in i:
        print(j)
        for k in j:
            print('\t', k)
verbnet.pprint(vnclass)

<Element 'MEMBER' at 0x7f4ca181a8b8>
<Element 'MEMBER' at 0x7f4ca181ae58>
<Element 'MEMBER' at 0x7f4ca181a138>
<Element 'THEMROLE' at 0x7f4ca181a4a8>
	 <Element 'SELRESTRS' at 0x7f4ca181ab88>
<Element 'FRAME' at 0x7f4ca181ab38>
	 <Element 'DESCRIPTION' at 0x7f4ca181af98>
	 <Element 'EXAMPLES' at 0x7f4ca1da8278>
	 <Element 'SYNTAX' at 0x7f4ca1da8a98>
	 <Element 'SEMANTICS' at 0x7f4ca17e4598>
<Element 'FRAME' at 0x7f4ca1809278>
	 <Element 'DESCRIPTION' at 0x7f4ca1809d68>
	 <Element 'EXAMPLES' at 0x7f4ca1809458>
	 <Element 'SYNTAX' at 0x7f4ca18091d8>
	 <Element 'SEMANTICS' at 0x7f4ca1809868>
<Element 'VNSUBCLASS' at 0x7f4ca1813868>
	 <Element 'MEMBERS' at 0x7f4ca1813548>
	 <Element 'THEMROLES' at 0x7f4ca1813ae8>
	 <Element 'FRAMES' at 0x7f4ca18138b8>
	 <Element 'SUBCLASSES' at 0x7f4ca17f5c78>


'exchange-13.6-1\n  Subclasses: exchange-13.6-1-1\n  Members: change exchange trade\n  Thematic roles:\n    * Location[+location -region]\n  Frames:\n    NP-PP (with-PP)\n      Example: One bell can swap places with another.\n      Syntax: NP[Theme1] VERB NP[Location] PREP[with] NP[Theme2]\n      Semantics:\n        * location(start(E), Theme1, Location_i)\n        * location(end(E), Theme1, Location_j)\n        * location(start(E), Theme2, Location_j)\n        * location(end(E), Theme2, Location_i)\n    Transitive (Location Object)\n      Example: The bells traded places.\n      Syntax: NP[Theme +plural] VERB NP[Location]\n      Semantics:\n        * location(start(E), Theme_i, Location_i)\n        * location(end(E), Theme_i, Location_j)\n        * location(start(E), Theme_j, Location_j)\n        * location(end(E), Theme_j, Location_i)'

# Answer Type Prediction

In [ ]:
if answer_type == 'HUMAN:group':
    

# SPARQL Query

In [6]:
sparql = SPARQLWrapper('http://dbpedia.org/sparql')

In [29]:
# PREFIX dbres: <http://dbpedia.org/resource/>

prefix = """
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX quepy: <http://www.machinalis.com/quepy#>
    PREFIX dbpedia: <http://dbpedia.org/ontology/>
    PREFIX dbpprop: <http://dbpedia.org/property/>
    PREFIX dbpedia-owl: <http://dbpedia.org/ontology/>
"""

In [48]:
query = prefix + """
    SELECT DISTINCT ?x2 WHERE {
      ?x0 rdf:type dbpedia-owl:Country.
      ?x0 rdfs:label "Bolivia"@en.
      ?x0 dbpedia-owl:capital ?x1.
      ?x1 rdfs:label ?x2.
    }
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
answer = sparql.query().convert()

In [50]:
if len(answer['results']['bindings']) != 0:
    for ans in answer['results']['bindings']:
        if ans['x2']['xml:lang'] == 'en':
            print(ans['x2']['value'])
else:
    print('No results returned')

Sucre


In [32]:
# PREFIX dbres: <http://dbpedia.org/resource/>

query = prefix + """
    SELECT DISTINCT * WHERE {
      ?url foaf:isPrimaryTopicOf wikipedia-en:asdfasdf.
    }
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
answer = sparql.query().convert()